## Load Packages

In [ ]:
## setup(1min)
# torch 버전 다운. torch>=2.0 에선 colosalai가 동작안함
!pip uninstall torch -y
!pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

# for transformers, 최신버전은 에러발생
!pip install transformers==4.35.2
!pip install accelerate==0.24.1

# for ColossalAI
!pip install colossalai==0.2.7

# setup data
!git clone https://github.com/airobotlab/KoChatGPT
!mv KoChatGPT/data_kochatgpt .
!mv KoChatGPT/img .

%cd KoChatGPT/colossalai_ChatGPT_230319/
!pip install .
%cd ../../

# setup library
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1

Found existing installation: torch 2.2.1+cu121
Uninstalling torch-2.2.1+cu121:
  Successfully uninstalled torch-2.2.1+cu121
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 1.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1+cu116 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.13.1+cu116 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 1.13.1+cu116 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 1.13.1+cu116 which is incompatible.
Torch version:1.13.1+cu116
cuda version: 11.6
cudnn version:8302
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 36.9 MB/s eta 0:00:00
     ━

Cloning into 'KoChatGPT'...
remote: Enumerating objects: 304, done.
remote: Total 304 (delta 0), reused 0 (delta 0), pack-reused 304
Receiving objects: 100% (304/304), 57.72 MiB | 12.12 MiB/s, done.
Resolving deltas: 100% (123/123), done.
/content/KoChatGPT/colossalai_ChatGPT_230319
Processing /content/KoChatGPT/colossalai_ChatGPT_230319
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# import
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from copy import deepcopy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
import pandas as pd
import argparse
import copy
import logging
import json
from dataclasses import dataclass, field

def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    """Collects the state dict and dump to disk."""
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {key: value.cpu() for key, value in list(state_dict.items())}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa

### 경로 설정

In [ ]:
# define argment
parser = argparse.ArgumentParser()
DATA_PATH = '/content/drive/MyDrive/KoMo/Dataset/total_minus.csv'
OUTPUT_PATH = '/content/drive/MyDrive/KoMo/SFT/SFT_0521'
parser.add_argument('--data_path_1_SFT', type=str, default=DATA_PATH)
parser.add_argument('--model_name', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--max_epochs', type=int, default=2)
parser.add_argument('--train_batch_size', type=int, default=8)
parser.add_argument('--output_dir', type=str, default=OUTPUT_PATH)

args = parser.parse_args(args=[])

# for test
args.model_name = 'skt/kogpt2-base-v2'  # SK GPT2, https://github.com/SKT-AI/KoGPT2
args.max_epochs = 5

print(args)

Namespace(data_path_1_SFT='/content/drive/MyDrive/KoMo/Dataset/total_minus.csv', model_name='skt/kogpt2-base-v2', max_epochs=5, train_batch_size=8, output_dir='/content/drive/MyDrive/KoMo/SFT/SFT_0521')


# Reward Model

## 환경설정

In [ ]:
## setup(1min)
# torch 버전 다운. torch>=2.0 에선 colosalai가 동작안함
!pip uninstall torch -y
!pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install loralib
!pip install wandb

import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

# for ColossalAI
!pip install colossalai==0.2.7

# setup data
!git clone https://github.com/airobotlab/KoChatGPT
!mv KoChatGPT/data_kochatgpt .
!mv KoChatGPT/img .

%cd KoChatGPT/colossalai_ChatGPT_230319/
!pip install .
%cd ../../

Found existing installation: torch 2.3.0+cu121
Uninstalling torch-2.3.0+cu121:
  Successfully uninstalled torch-2.3.0+cu121
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 1.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 1.13.1+cu116 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 1.13.1+cu116 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 1.13.1+cu116 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 34.5 MB/s eta 0:0

Cloning into 'KoChatGPT'...
remote: Enumerating objects: 304, done.
remote: Total 304 (delta 0), reused 0 (delta 0), pack-reused 304
Receiving objects: 100% (304/304), 57.72 MiB | 12.80 MiB/s, done.
Resolving deltas: 100% (123/123), done.
/content/KoChatGPT/colossalai_ChatGPT_230319
Processing /content/KoChatGPT/colossalai_ChatGPT_230319
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### pip install 후 해당 경로 파일 수정 -> 런타임 재시작(이후엔 pip install X import만 O)

##### trainng시, tqdm 출력으로 인한 크롬 정지 해결

/usr/local/lib/python3.10/dist-packages/chatgpt/trainer/rm.py 파일 아래와 같이 수정 후, 세션 재시작

```
from abc import ABC

import loralib as lora
import torch
from chatgpt.dataset import RewardDataset
from chatgpt.models.loss import PairWiseLoss
from torch.optim import Adam, Optimizer
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import os

from .strategies import Strategy
from .utils import is_rank_0
import wandb


class RewardModelTrainer(ABC):
    """
        Trainer to use while training reward model.

    Args:
        model (torch.nn.Module): the model to train
        strategy (Strategy): the strategy to use for training
        optim(Optimizer): the optimizer to use for training
        train_dataset (RewardDataset): the dataset to use for training
        eval_dataset (RewardDataset): the dataset to use for evaluation
        batch_size (int, defaults to 1): the batch size while training
        max_epochs (int, defaults to 2): the number of epochs to train
        optim_kwargs (dict, defaults to {'lr':1e-4}): the kwargs to use while initializing optimizer
    """

    def __init__(
        self,
        model,
        strategy: Strategy,
        optim: Optimizer,
        train_dataset: RewardDataset,
        eval_dataset: RewardDataset,
        batch_size: int = 1,
        max_epochs: int = 2,
    ) -> None:
        super().__init__()
        self.strategy = strategy
        self.epochs = max_epochs
        self.train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
        self.eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size)

        self.model = strategy.setup_model(model)
        if "DDP" in str(self.strategy):
            self.model = self.model.module
        self.loss_fn = PairWiseLoss()
        self.optimizer = strategy.setup_optimizer(optim, self.model)

    def fit(self, use_lora, RM_OUTPUT_DIR):
        #epoch_bar = tqdm(range(self.epochs), desc='Train epoch', disable=not is_rank_0())
        print("training start!")

        for epoch in tqdm(range(self.epochs)):
            #step_bar = tqdm(range(self.train_dataloader.__len__()),
            #                desc='Train step of epoch %d' % epoch,
            #                disable=not is_rank_0())
            # train
            self.model.train()
            for chosen_ids, c_mask, reject_ids, r_mask in tqdm(self.train_dataloader):
                chosen_ids = chosen_ids.squeeze(1).cuda()
                c_mask = c_mask.squeeze(1).cuda()
                reject_ids = reject_ids.squeeze(1).cuda()
                r_mask = r_mask.squeeze(1).cuda()
                chosen_reward = self.model(chosen_ids, attention_mask=c_mask)
                reject_reward = self.model(reject_ids, attention_mask=r_mask)
                loss = self.loss_fn(chosen_reward, reject_reward)
                wandb.log({"training_loss":loss})
                self.strategy.backward(loss, self.model, self.optimizer)
                self.strategy.optimizer_step(self.optimizer)
                self.optimizer.zero_grad()
                #step_bar.update()
                #step_bar.set_postfix({'loss': loss.item()})

            # eval
            self.model.eval()
            with torch.no_grad():
                dist = 0
                loss_sum = 0
                for chosen_ids, c_mask, reject_ids, r_mask in self.eval_dataloader:
                    chosen_ids = chosen_ids.squeeze(1).cuda()
                    c_mask = c_mask.squeeze(1).cuda()
                    reject_ids = reject_ids.squeeze(1).cuda()
                    r_mask = r_mask.squeeze(1).cuda()
                    chosen_reward = self.model(chosen_ids, attention_mask=c_mask)
                    reject_reward = self.model(reject_ids, attention_mask=r_mask)
                    dist += (chosen_reward - reject_reward).mean().item()
                    loss = self.loss_fn(chosen_reward, reject_reward)
                    loss_sum += loss.item()
                dist_mean = dist / self.eval_dataloader.__len__()
                loss_mean = loss_sum / self.eval_dataloader.__len__()
                wandb.log({"validation_loss":loss_mean})
                wandb.log({"dist_mean":dist_mean})
                wandb.log({"loss_mean":loss_mean})
            #epoch_bar.update()
            #step_bar.set_postfix({'loss': loss_mean, 'dist_mean': dist_mean})
            #step_bar.close()


            # 모델 저장
            RM_MODEL_NAME = os.path.join(RM_OUTPUT_DIR, f'RM_model_epoch{epoch + 1}_{round(loss_mean, 5)}.pt')
            RM_OPTIM_NAME = os.path.join(RM_OUTPUT_DIR, f'RM_optim_epoch{epoch + 1}_{round(loss_mean, 5)}.pt')
            self.strategy.save_model(self.model, RM_MODEL_NAME, only_rank0=False)
            self.strategy.save_optimizer(self.optimizer, RM_OPTIM_NAME, only_rank0=False)
```

##### MLP layer 수정 후, 발생하는 에러 해결
/usr/local/lib/python3.10/dist-packages/chatgpt/models/base/reward_model.py 수정

```
from typing import Optional

import torch
import torch.nn as nn

from ..lora import LoRAModule


class RewardModel(LoRAModule):
    """
    Reward model base class.

    Args:
        model (nn.Module): Reward model.
        value_head (nn.Module): Value head to get reward score.
        lora_rank (int): LoRA rank.
        lora_train_bias (str): LoRA bias training mode.
    """

    def __init__(self,
                 model: nn.Module,
                 value_head: Optional[nn.Module] = None,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none') -> None:
        super().__init__(lora_rank=lora_rank, lora_train_bias=lora_train_bias)
        self.model = model
        self.convert_to_lora()

        if value_head is not None:
            # if value_head.out_features != 1:
            #     print("Out feature should be 1")
                # raise ValueError("The value head of reward model's output dim should be 1!")
            self.value_head = value_head
        else:
            self.value_head = nn.Linear(model.config.n_embd, 1)

    def forward(self, sequences: torch.LongTensor, attention_mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        outputs = self.model(sequences, attention_mask=attention_mask)
        last_hidden_states = outputs['last_hidden_state']
        values = self.value_head(last_hidden_states)[:, :-1]
        value = values.mean(dim=1).squeeze(1)    # ensure shape is (B)
        return value

```

### SFT, RM Load

In [ ]:
# import
import argparse

import loralib as lora
import torch
torch.cuda.empty_cache()
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.models.bloom import BLOOMRM
from chatgpt.models.gpt import GPTRM
from chatgpt.models.opt import OPTRM
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import ColossalAIStrategy, DDPStrategy, NaiveStrategy
from datasets import load_dataset
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

from colossalai.nn.optimizer import HybridAdam

import os
import json

# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"

RM_MODEL_OUTPUT_PATH = '/content/drive/MyDrive/KoMo/RM'
RM_DATASET_PATH = '/content/drive/MyDrive/KoMo/Dataset/total_minus.json'
SFT_MODEL_NAME = '/content/drive/MyDrive/KoMo/SFT/0521_25628_15epoch/checkpoint-3204'

# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--output_dir', type=str, default=RM_MODEL_OUTPUT_PATH)
parser.add_argument('--data_path_2_RM', type=str, default=RM_DATASET_PATH)
parser.add_argument('--strategy',
                    choices=['naive', 'ddp', 'colossalai_gemini', 'colossalai_zero2'],
                    default='naive')
parser.add_argument('--model', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--dataset', type=str, default='Dahoas/rm-static')
parser.add_argument('--save_path', type=str, default='rm_ckpt.pth')
parser.add_argument('--max_epochs', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=4)
parser.add_argument('--lora_rank', type=int, default=0, help="low-rank adaptation matrices rank")
# parser.add_argument('--max_len', type=int, default=512)  # wygo 추가
parser.add_argument('--max_len', type=int, default=64)
args = parser.parse_args(args=[])

# for test
args.max_epochs = 5
args.pretrain = SFT_MODEL_NAME  # pretrained 모델 가져오기
args.verbose = True

print(args)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

/usr/local/lib/python3.10/dist-packages/torch/library.py:130: UserWarning: Overriding a previously registered kernel for the same operator and the same dispatch key
  operator: aten::index.Tensor(Tensor self, Tensor?[] indices) -> Tensor
    registered at aten/src/ATen/RegisterSchema.cpp:6
  dispatch key: Meta
  previous kernel: registered at ../aten/src/ATen/functorch/BatchRulesScatterOps.cpp:1053
       new kernel: registered at /dev/null:241 (Triggered internally at ../aten/src/ATen/core/dispatch/OperatorEntry.cpp:150.)
  self.m.impl(name, dispatch_key, fn)


Namespace(output_dir='/content/drive/MyDrive/KoMo/RM', data_path_2_RM='/content/drive/MyDrive/KoMo/Dataset/total_minus.json', strategy='naive', model='gpt2', pretrain='/content/drive/MyDrive/KoMo/SFT/0521_25628_15epoch/checkpoint-3204', dataset='Dahoas/rm-static', save_path='rm_ckpt.pth', max_epochs=5, batch_size=4, lora_rank=0, max_len=64, verbose=True)


In [ ]:
os.environ['RANK'] = '0'
os.environ['LOCAL_RANK'] = '0'
os.environ['WORLD_SIZE'] = '2'
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '42043'

In [ ]:
# configure strategy
if args.strategy == 'naive':
    strategy = NaiveStrategy()
elif args.strategy == 'ddp':
    strategy = DDPStrategy()
elif args.strategy == 'colossalai_gemini':
    strategy = ColossalAIStrategy(stage=3, placement_policy='cuda')
elif args.strategy == 'colossalai_zero2':
    strategy = ColossalAIStrategy(stage=2, placement_policy='cuda')
else:
    raise ValueError(f'Unsupported strategy "{args.strategy}"')

In [ ]:
# customizing, https://github.com/hpcaitech/ColossalAI/blob/2e16f842a9e5b1fb54e7e41070e9d2bb5cd64d7c/applications/ChatGPT/chatgpt/nn/gpt_rm.py#L29
from typing import Optional

import torch.nn as nn
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

# from ..base import RewardModel
from chatgpt.models.base import RewardModel


class GPTRM_custom(RewardModel):
    """
    GPT Reward model.
    Args:
        pretrained (str): Pretrained model name or path.
        config (GPT2Config): Model config.
        checkpoint (bool): Enable gradient checkpointing.
        lora_rank (int): Rank of the low-rank approximation.
        lora_train_bias (str): LoRA bias training mode.
    """

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            print("koGPT2 model load success!")
            model.resize_token_embeddings(len(tokenizer))  # wygo 추가!!!
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())

        if checkpoint:
            model.gradient_checkpointing_enable()


        # model = model.resize_token_embeddings(len(tokenizer))
        """
        #original MLP 1 layer
        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)
        """

        ## MLP 2 Layer
        """
        value_head = nn.Sequential(
            nn.Linear(model.config.n_embd, model.config.n_embd // 2),
            nn.ReLU(),
            nn.Linear(model.config.n_embd // 2, 1)
        )
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        """
        ## MLP 3 Layer
        value_head = nn.Sequential(
            nn.Linear(model.config.n_embd, model.config.n_embd // 4),
            nn.ReLU(),
            nn.Linear(model.config.n_embd // 4, model.config.n_embd // 2),
            nn.ReLU(),
            nn.Linear(model.config.n_embd // 2, 1)
        )
        super().__init__(model, value_head, lora_rank, lora_train_bias)


        # 추가, 230421
        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained
            print("pretrained model load success!")

    # 추가, 230421, config.json을 생성하기 위해 추가
    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [ ]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"

# configure model, tokenizer
with strategy.model_init_context():
    # load pretrained gpt2
    if args.model == 'gpt2':
#         tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        # tokenizer = AutoTokenizer.from_pretrained(args.pretrain)

        tokenizer = AutoTokenizer.from_pretrained(
            'skt/kogpt2-base-v2',
            padding_side="right",
            model_max_length=512,
        )
        tokenizer.add_special_tokens(
            {
                "eos_token": DEFAULT_EOS_TOKEN,
                "bos_token": DEFAULT_BOS_TOKEN,
                "unk_token": DEFAULT_UNK_TOKEN,
            }
        )
        tokenizer.pad_token = tokenizer.eos_token
        model = GPTRM_custom(pretrained=args.pretrain, lora_rank=args.lora_rank, tokenizer=tokenizer).cuda() #.cuda() 붙여서 실행하기

    elif args.model == 'bloom':
        model = BLOOMRM(pretrained=args.pretrain, lora_rank=args.lora_rank).cuda()
        tokenizer = BloomTokenizerFast.from_pretrained(args.pretrain)

    elif args.model == 'opt':
        model = OPTRM(pretrained=args.pretrain, lora_rank=args.lora_rank).cuda()
        tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

    else:
        raise ValueError(f'Unsupported model "{args.model}"')


    # model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


koGPT2 model load success!
pretrained model load success!


## RM training

#### Prepare train dataset



```
    {	// 우리 json 형식
        "prompt": "욕설 문장[SEP]",
        "chosen": "순화 + 이모지[SEP]",	// rank 0
        "rejected": "욕설[SEP]"			// rank 2
    },

    {
        "prompt": "욕설 문장[SEP]",
        "chosen": "순화[SEP]",			// rank 1
        "rejected": "욕설[SEP]"			// rank 2
    },

    {
        "prompt": "욕설 문장[SEP]",
        "chosen": "순화 + 이모지[SEP]",	// rank 0
        "rejected": "순화[SEP]",			// rank 1
    }
```



In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

DATA_PATH = '/content/drive/MyDrive/KoMo/Dataset/datast_30000_ranking.json'

In [ ]:
import json

# make ranking data to chosen, rejetced data
# with open(args.data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
with open(DATA_PATH, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    # if args.verbose:
    print('## data check ##')
    print((list_data_dict[0]))
    print(len(list_data_dict[10000:]))

total_data_ranking2chosen = []
for tmp in list_data_dict[10000:]:
    one_data_ranking2chosen = []

    """

    Rank

    Dataset                                     Generated Dataset
    0 : Purification & Emojis               |   일상 데이터셋 + GPT3.5 Turbo로 이모지 생성
    1 : only Purification                   |   일상 데이터셋
    2 : Poor Purification & Poor Emojis     |   일상 데이터셋에 Claude로 욕설/혐오 표현 생성 & 말투 바꿈

    """


    # Ver2에서 주석처리
    # data 1) 0 VS 1

    print("data 0 vs 1 INCLUDED!")
    data = {}
    data['prompt'] = tmp['completion_2']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = str(tmp['completion_0'])
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = str(tmp['completion_1'])
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)


    # data 2) 0 VS 2
    data = {}
    data['prompt'] = tmp['completion_2']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = str(tmp['completion_0'])
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = str(tmp['completion_2'])
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # data 1) 1 VS 2
    data = {}
    data['prompt'] = tmp['completion_2']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = str(tmp['completion_1'])
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = str(tmp['completion_2'])
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])
print('data example: \n%s'%total_data_ranking2chosen[55])
print('data example: \n%s'%total_data_ranking2chosen[65])

Streaming output truncated to the last 5000 lines.
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
data 0 vs 1 INCLUDED!
dat

In [ ]:
# prepare for data and dataset
import random
import math

random.seed(230319)
random.shuffle(total_data_ranking2chosen)

data_len = len(total_data_ranking2chosen)
train_len = math.floor(int(data_len * 0.8))

train_data = total_data_ranking2chosen[:train_len]
eval_data = total_data_ranking2chosen[train_len:]

#test
# train_data = total_data_ranking2chosen[:20]
# eval_data = total_data_ranking2chosen[20:40]

train_dataset = None
eval_dataset = None

train_dataset = RewardDataset(train_data, tokenizer, args.max_len)
eval_dataset = RewardDataset(eval_data, tokenizer, args.max_len)


100%|██████████| 11982/11982 [00:05<00:00, 2101.75it/s]


### Load RM

In [ ]:
import torch
from chatgpt.models import RewardModel

def load_checkpoint_RM(model_class, model, model_path, optimizer_class=None, optimizer_path=None, device = 'cuda'):
    """
    Load model and optimizer from checkpoint.

    Args:
        model_class: the class of the model to load
        model_path (str): the path to the saved model checkpoint
        optimizer_class: the class of the optimizer to load (default: None)
        optimizer_path (str): the path to the saved optimizer checkpoint (default: None)
        device (str): the device to load the model onto (default: 'cpu')

    Returns:
        model: the loaded model
        optimizer: the loaded optimizer (if optimizer_path is provided)
    """

    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint)

    optimizer = None
    if optimizer_class and optimizer_path:
        optimizer = optimizer_class(model.parameters())
        checkpoint = torch.load(optimizer_path, map_location=device)
        optimizer.load_state_dict(checkpoint)

    return model, optimizer

In [ ]:
from torch.optim import Adam

RM_MODEL_PATH = '/content/drive/MyDrive/KoMo/RM/0527_0317/RM_model_epoch1_0.04957.pt'
RM_OPTIM_PATH = '/content/drive/MyDrive/KoMo/RM/0527_0317/RM_optim_epoch1_0.04957.pt'

# Load the model and optimizer
model, optimizer = load_checkpoint_RM(RewardModel, model, RM_MODEL_PATH, Adam, RM_OPTIM_PATH, device='cuda')

### Train

In [ ]:
from torch.optim import Adam

def train_RM(lr, batch_size, epochs, RM_OUTPUT_DIR):
    batch_size = batch_size
    epochs = epochs
    RM_OUTPUT_DIR = RM_OUTPUT_DIR
    print("###\n learning rate", lr)
    print("batch size", batch_size)
    print("epochs", epochs, '\n')
    optim = Adam(model.parameters(), lr=lr)
    trainer = RewardModelTrainer(model=model,
                                strategy=strategy,
                                optim=optim,
                                train_dataset=train_dataset,
                                eval_dataset=eval_dataset,
                                batch_size=batch_size,    #  batch_size=args.batch_size,
                                max_epochs=epochs)    # args.max_epochs


    RM_MODEL_NAME = f'RM_model_epoch{epochs}.pt'
    RM_OPTIM_NAME = f'RM_optim_epoch{epochs}.pt'


    trainer.fit(use_lora=args.lora_rank, RM_OUTPUT_DIR = RM_OUTPUT_DIR)

    ## save
    # save model checkpoint after fitting on only rank0
    strategy.save_model(model, os.path.join(RM_OUTPUT_DIR, RM_MODEL_NAME), only_rank0=False)
    # save optimizer checkpoint on all ranks
    strategy.save_optimizer(optim, os.path.join(RM_OUTPUT_DIR, RM_OPTIM_NAME), only_rank0=False)

    model.save_pretrained(RM_OUTPUT_DIR)  # config.json 생성

In [ ]:
import os
from datetime import datetime
import pytz

kst = pytz.timezone('Asia/Seoul')
current_time = datetime.now(kst)

time_str = current_time.strftime('%m%d_%H%M')
RM_OUTPUT_DIR = f'/content/drive/MyDrive/KoMo/RM/{time_str}/'
os.makedirs(RM_OUTPUT_DIR, exist_ok=True)

batch_size = 4

STEP = 3
lr_list = [5e-5, 1e-5, 1e-6]
epochs_list = [5, 1, 1]

for i in range(STEP):
    train_RM(lr_list[i], batch_size, epochs_list[i], RM_OUTPUT_DIR)

###
 learning rate 5e-05
batch size 4
epochs 5 

training start!


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/11982 [00:00<?, ?it/s]

  0%|          | 0/11982 [00:00<?, ?it/s]

  0%|          | 0/11982 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/11982 [00:00<?, ?it/s]

###
 learning rate 1e-06
batch size 4
epochs 1 

training start!


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/11982 [00:00<?, ?it/s]

## RM Inference

In [ ]:
def inference_RM(input_text='인공지능은 인공지능 입니다', print_log = True):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    if print_log:
        print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

In [ ]:
print("### Profanity")
inference_RM(input_text='에휴 알파카 새끼 존나 싫어 ㅅㅂ')
inference_RM(input_text='에휴 알파카 새끼 존나 싫어 ㅅㅂ 이름만 들어도 환멸나네 이 돼지새끼')
print("\n")
print("### Daily Sentences")
inference_RM(input_text='알파카 나 너 사랑하냐?')
inference_RM(input_text='알파카 사랑해')
print("\n")
print("### w/ Emojis")
inference_RM(input_text='알파카 나 너 사랑😍하냐?')
inference_RM(input_text='알파카 나 너 사랑😍하냐? 내가 이렇게 너를 좋아하게 될 줄이야.')
inference_RM(input_text='알파카 사랑😍해')
inference_RM(input_text='알파카 사랑💔해')
inference_RM(input_text='알파카🦙 사랑😍해')
print("\n")

print("-"*20)
print("### Profanity")
inference_RM(input_text='아 존나 홍어 냄새 여기까지 나네 ㅋㅋ 엠생이노')
inference_RM(input_text='아 ㅅㅂ 홍어 냄새 라도냐?')
inference_RM(input_text='아 ㅅㅂ 홍어🐟 냄새 라도냐?')
inference_RM(input_text='아 ㅅㅂ 홍어🐟 냄새 너 씨발 라도냐?')
print("\n")
print("### Daily Sentences")
inference_RM(input_text='혹시 전라도 사람이세요?')
inference_RM(input_text='혹시 전라도 사람이세요? 뭔가 그쪽에서 태어나신거같아요. 맞나요?')
print("\n")
print("### w/ Emojis")
inference_RM(input_text='혹시 전라도 사람🧑🏻이세요?')
inference_RM(input_text='혹시 전라도 사람🧑🏻이세요? 뭔가 그쪽에서 태어나신거같아요. 맞나요?')
print("\n")

print("-"*20)
print("### Profanity")
inference_RM(input_text='게이 새끼들 다 정상으로 돌아오게 치료받자')
inference_RM(input_text='게이 새끼들 다 정상으로 돌아오게 치료받자 역겹노')
inference_RM(input_text='ㅅㅂ 게이 새끼들 다 정상으로 돌아오게 치료받자 역겹노')
inference_RM(input_text='난 게이가 이해가 안 돼')
inference_RM(input_text='난 시발 게이 새끼들이 존나 이해가 안 돼')
inference_RM(input_text='난 시1발 게이 새끼들이 존~~나게 이해가 안 돼 ㅋㅋ 왜 그지랄하는지 모르겠다. 내가 이해해줘야하나?')

print("\n")
print("### Daily Sentences")
inference_RM(input_text='동성애 응원합니다!')
inference_RM(input_text='동성애 응원합니다! 충분히 응원받아야 마땅하다고 생각해요!')
inference_RM(input_text='동성애 응원합니다! 충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요!')
inference_RM(input_text='동성애 응원합니다! 충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요! 편견에 찬 시선으로 바라보지 맙시다.')

print("\n")
print("### w/ Emojis")
inference_RM(input_text='동성애🌈 응원합니다!')
inference_RM(input_text='동성애🌈 응원합니다! 충분히 응원받아야 마땅하다고 생각해요!')
inference_RM(input_text='동성애🌈 응원합니다! 충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요!')
inference_RM(input_text='동성애🌈 응원합니다! 충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요! 편견에 찬 시선으로 바라보지 맙시다.')
print("\n### w/ Wrong emojis")
inference_RM(input_text='동성애 응원합니다!👎🏻 충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요!')
inference_RM(input_text='동성애 응원합니다!💔 충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요!')
inference_RM(input_text='동성애 응원합니다!💩  충분히 응원받아야 마땅하다고 생각해요! 전혀 이상하지않아요!')

print("-"*20)
print("### Profanity")
inference_RM(input_text='아 ㅅㅂ 게임 ㅈ같이하네 아니 거기서 왜 궁을 쏘냐고 ㅂㅅ이냐?')
print("\n")
print("### Daily Sentences")
inference_RM(input_text='게임 실력을 늘리셔야 할 거 같아요. 아직은 부족해 보여요')
print("\n")
print("### Cleaned Sentences")
inference_RM(input_text='게임 실력을 늘리셔야 할 거 같아요😢. 아직은 부족해 보여요')
print("-"*20)

### Profanity
input: 에휴 알파카 새끼 존나 싫어 ㅅㅂ
reward score: -8.5
input: 에휴 알파카 새끼 존나 싫어 ㅅㅂ 이름만 들어도 환멸나네 이 돼지새끼
reward score: -8.3


### Daily Sentences
input: 알파카 나 너 사랑하냐?
reward score: -8.6
input: 알파카 사랑해
reward score: -9.4


### w/ Emojis
input: 알파카 나 너 사랑😍하냐?
reward score: 12.9
input: 알파카 나 너 사랑😍하냐? 내가 이렇게 너를 좋아하게 될 줄이야.
reward score: 1.0
input: 알파카 사랑😍해
reward score: 31.4
input: 알파카 사랑💔해
reward score: 8.9
input: 알파카🦙 사랑😍해
reward score: 31.7


--------------------
### Profanity
input: 아 존나 홍어 냄새 여기까지 나네 ㅋㅋ 엠생이노
reward score: -8.5
input: 아 ㅅㅂ 홍어 냄새 라도냐?
reward score: -8.4
input: 아 ㅅㅂ 홍어🐟 냄새 라도냐?
reward score: -1.7
input: 아 ㅅㅂ 홍어🐟 냄새 너 씨발 라도냐?
reward score: -3.7


### Daily Sentences
input: 혹시 전라도 사람이세요?
reward score: -8.2
input: 혹시 전라도 사람이세요? 뭔가 그쪽에서 태어나신거같아요. 맞나요?
reward score: -8.2


### w/ Emojis
input: 혹시 전라도 사람🧑🏻이세요?
reward score: 11.8
input: 혹시 전라도 사람🧑🏻이세요? 뭔가 그쪽에서 태어나신거같아요. 맞나요?
reward score: -0.7


--------------------
### Profanity
input: 게이 새끼들 다 정상으로 돌아오게 치료받자
reward score: -8.